In [1]:
with open("dataset.txt") as f:
    text = f.read()

In [4]:
print("Data length:", len(text))

Data lengt: 1115394


In [6]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


#### Create an ecoding of individual characters

In practice, we do not want to encode every charcter or every word, something in between

For example: use BPE or byte pair encoding
- More tokens -> shorter encodings
- Less tokens -> longer encodings

In [9]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("Vocab:", "".join(chars))
print("Vocab size:", vocab_size)

Vocab: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size: 65


In [12]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(decode(encode("Hello, how are you?")))
print(encode("Hello, how are you?"))

Hello, how are you?
[20, 43, 50, 50, 53, 6, 1, 46, 53, 61, 1, 39, 56, 43, 1, 63, 53, 59, 12]


### Encode entire dataset

In [14]:
import torch

In [17]:
# Tensor is a pytorch matrix, use 64-bit integers
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])
# 0 is new line, 1 is space

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [19]:
# split dataset into 90% train, 10% validation
# if we trained on entire dataset, it would just spit out exactly what was put in
sp = int(len(data) * 0.9)
train_data = data[:sp]
val_data = data[sp:]

In [21]:
# train on chunks of tokens at a time, lets do 8
block_size = 8
train_data[:block_size + 1]

# We will train in parallel

# for our block: [18, 47, 56, 57, 58,  1, 15, 47, 58]

# we have 8 training data points
# [18] -> 47
# [18, 47] -> 56
# [18, 47, 56] -> 57
# ...
# [18, 47, 56, 57, 58,  1, 15, 47] -> 58

# this is the time dimension of our inputs!

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [28]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for i in range(block_size):
    context = x[:i + 1]
    target = y[i]
    print(context, "->", target)

# we want our transformer to know how to predict from context 1 to blocksize
# after blocksize of context, we need to start truncating 

tensor([18]) -> tensor(47)
tensor([18, 47]) -> tensor(56)
tensor([18, 47, 56]) -> tensor(57)
tensor([18, 47, 56, 57]) -> tensor(58)
tensor([18, 47, 56, 57, 58]) -> tensor(1)
tensor([18, 47, 56, 57, 58,  1]) -> tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) -> tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) -> tensor(58)


In [40]:
# for gpu efficency, we can also batch chunks together (batch dimension)

torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel
block_size = 8 # maximum context lenght for prediction

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size - 1, (batch_size,)) # get batch_size random numbers for start indexes of our blocks
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size + 1] for i in ix])
    # for each index 
    # context = x[batch_index][:index + 1]
    # target = y[index]
    return x,y

xb, yb = get_batch("train")
print(xb.shape, yb.shape)
print(xb)
print(yb)

torch.Size([4, 8]) torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])


### Bigram Language Model
https://web.stanford.edu/~jurafsky/slp3/3.pdf

n-gram model - a probabilistic model that estimates the probability of a word given n-1 previous words


In [46]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
print(xb)

tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])


In [48]:
class BigramLanguageModel(nn.Module):
    def __init__ (self, vocab_size):
        super().__init__()
        # create a vocab_size x vocab_size lookup table for probabilities
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits =logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idk is (B, T) array of indices in current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss)

torch.Size([32, 65]) tensor(4.7525, grad_fn=<NllLossBackward0>)


In [53]:
# totally random model
idx = torch.zeros((1,1), dtype=torch.long) # [0] as input (newline char)
print(decode(m.generate(idx, max_new_tokens=100,)[0].tolist()))


NO.X,DELunD'Bjrk:V$b p?UwM$oljzZHXCj? U$IxMkKDzRkY eJulQosKGr'jIi:'zBL&,ojkYzgZIiULYb
!TzBwxWBs3j3&u


### Train the model 
Using AdamW optimizer
- Optimizer using exponentially weighted average of the gradients

smaller models can use larger learning rates

In [54]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [61]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')

    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4519717693328857


In [63]:
# result after training 
idx = torch.zeros((1,1), dtype=torch.long) # [0] as input (newline char)
print(decode(m.generate(idx, max_new_tokens=300,)[0].tolist()))


IUCico s s tithe.
Of hary, chethe.

DUEd mod rrd mipy tee sourdine; rand toemystot tur d imasld ho ma oand hat g:
LAMAnd ud or inatsa thaisire!
ICUSha!
D owinknthis Wh t n aw w, chou t thare auntot hon she rthtamiveswhit llapank'fowimy hesmy har,

LI
Wowithardire.
By theexpo.
H:
QUnr merod th ch oul
